In [1]:
import pandas as pd
import re
import numpy as np

# Подготовка данных с информацией аниме

In [2]:
anime = pd.read_csv('/media/vaserkn/HDD/Anime_rec_sys/anime.csv')

In [3]:
for col in anime.columns:
        anime.loc[anime[col] == 'Unknown', col] = np.nan

## Найдем рейтинг для строк, где находится np.nan, с помощью 'Score-1', 'Score-2',..., 'Score-10', если это возможно

In [4]:
#  Создание массива с колонками 'Score-1',...,'Score-10' для удобства
score_cols = []
for col in anime.columns:
    if 'Score-' in col:
        score_cols.append(col)
score_cols.reverse()
score_cols

['Score-1',
 'Score-2',
 'Score-3',
 'Score-4',
 'Score-5',
 'Score-6',
 'Score-7',
 'Score-8',
 'Score-9',
 'Score-10']

In [5]:
#  Выберем колонки только связанные с рейтингом и с np.nan
y = anime.loc[anime['Score'].isnull()]
y = pd.concat([y[['MAL_ID', 'Score']], y[score_cols]], axis=1)
y.isnull().sum()

MAL_ID         0
Score       5141
Score-1      452
Score-2     1570
Score-3     1299
Score-4      974
Score-5      584
Score-6      511
Score-7      503
Score-8     1347
Score-9     2921
Score-10     437
dtype: int64

In [6]:
#  Удаляем все колонки 'Score-1',...,'Score-2' с np.nan и меняем тип оставшихся на float
y.loc[:, 'Score'] = 0
y = y.dropna()
y.loc[:, 'Score'] = np.NaN
for col in score_cols:
    y[col] = y[col].astype(float)
y.isnull().sum()


MAL_ID         0
Score       1320
Score-1        0
Score-2        0
Score-3        0
Score-4        0
Score-5        0
Score-6        0
Score-7        0
Score-8        0
Score-9        0
Score-10       0
dtype: int64

In [7]:
# Находим ср.рейтинг и записываем его в dataset
for i in range(y.shape[0]):
    total_score, total_users = 0, 0
    for score, col in enumerate(score_cols):
        total_score += y.iloc[i][col] * (score + 1)
        total_users += y.iloc[i][col]
    MAL_ID = y.iloc[i]['MAL_ID']
    anime.loc[anime['MAL_ID'] == MAL_ID, 'Score'] = round(total_score / total_users, 2)

In [8]:
# Удалось найти 1320 рейтингов
anime.isnull().sum()

MAL_ID               0
Name                 0
Score             3821
Genres              63
English name     10565
Japanese name       48
Type                37
Episodes           516
Aired              309
Premiered        12817
Producers         7794
Licensors        13616
Studios           7079
Source            3567
Duration           555
Rating             688
Ranked            1762
Popularity           0
Members              0
Favorites            0
Watching             0
Completed            0
On-Hold              0
Dropped              0
Plan to Watch        0
Score-10           437
Score-9           3167
Score-8           1371
Score-7            503
Score-6            511
Score-5            584
Score-4            977
Score-3           1307
Score-2           1597
Score-1            459
dtype: int64

## Отбор колонок

In [9]:
anime.columns

Index(['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
      dtype='object')

In [10]:
selected_cols = ['MAL_ID', 'Name', 'Score', 'Genres', 'Type', 'Episodes', 'Aired', 'Duration', 'Rating', 
                 'Popularity', 'Members', 'Favorites']

In [11]:
anime = anime[selected_cols]

In [12]:
anime['Score'] = anime['Score'].astype(float)
anime['MAL_ID'] = anime['MAL_ID'].astype(object)

## Заполним пропущенные значения в рейтинге средним рейтингом всех аниме

In [13]:
anime.loc[:, 'Score'] = anime.loc[:, 'Score'].fillna(anime['Score'].mean())

## Отчистим названия аниме от бесполезных символов

In [16]:
def clean(text):
    text = re.sub(r'&quot;', '', text)
    text = re.sub(r'.hack//', '', text)
    text = re.sub(r'&#039;', '', text)
    text = re.sub(r'A&#039;s', '', text)
    text = re.sub(r'I&#039;', 'I\'', text)
    text = re.sub(r'&amp;', 'and', text)
    
    return text

In [26]:
anime['Name'] = anime['Name'].apply(clean)

In [18]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17562 entries, 0 to 17561
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MAL_ID      17562 non-null  object 
 1   Name        17562 non-null  object 
 2   Score       17562 non-null  float64
 3   Genres      17499 non-null  object 
 4   Type        17525 non-null  object 
 5   Episodes    17046 non-null  object 
 6   Aired       17253 non-null  object 
 7   Duration    17007 non-null  object 
 8   Rating      16874 non-null  object 
 9   Popularity  17562 non-null  float64
 10  Members     17562 non-null  float64
 11  Favorites   17562 non-null  float64
dtypes: float64(4), object(8)
memory usage: 1.6+ MB


In [19]:
anime.isnull().sum()

MAL_ID          0
Name            0
Score           0
Genres         63
Type           37
Episodes      516
Aired         309
Duration      555
Rating        688
Popularity      0
Members         0
Favorites       0
dtype: int64

In [21]:
anime.to_csv('anime.csv')

# Подготовка данных с информацией о рейтинге

In [2]:
ratings = pd.read_csv('/media/vaserkn/HDD/Anime_rec_sys/rating_complete.csv')

## Возьмём миллион значений, чтобы хватало памяти

In [12]:
ratings = ratings.iloc[:1000000]

In [13]:
x = ratings['user_id'].value_counts()

## Отберём пользователей, поставивших рейтинги 10 и больше аниме

In [14]:
raitngs = ratings[ratings['user_id'].isin(x[x >= 10].index)]

In [15]:
ratings.to_csv('ratings.csv')